## Evaluate Yolov3 detector against kitti dataset
In this notebook we aim to provide performance metrics for the YoloV3 detector by leveraging the kitti dataset videos and their ground truth labels. The procedure is as follows:  
1) Load Yolov3 detector from the Einstein project.  
2) Perform object detection on kitti video.  
3) Evaluate accuracy of detections against kitti ground truth labels for given video.  
#### Note:  
We are using python from `/home/robert/PycharmProjects/Einstein/venv/bin/python`  
## See public benchmark submissions here:
http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d  
## PASCAL criterion paper:
http://host.robots.ox.ac.uk/pascal/VOC/voc2010/devkit_doc_08-May-2010.pdf  (See section 4.4)  
## Official KITTI Object Detection Evaluation scripts:
https://s3.eu-central-1.amazonaws.com/avg-kitti/devkit_object.zip  
Also saved locally in this folder at `./devkit_object/`  
See `./devkit_object/cpp/evaluate_object.cpp` for official evaluation code. 

## 1) Load Yolov3 detector from the Einstein project: 

In [1]:
import sys
sys.path.append('/home/robert/PycharmProjects/Einstein/src')
sys.path

['',
 '/home/robert/caffe-0.15.9/python',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/home/robert/PycharmProjects/Bob/venv/lib/python3.5/site-packages',
 '/home/robert/PycharmProjects/Bob/venv/lib/python3.5/site-packages/IPython/extensions',
 '/home/robert/.ipython',
 '/home/robert/PycharmProjects/Einstein/src']

In [2]:
from detection.car_detector_tf_v2 import CarDetectorTFV2
from detection.car_detector_tf import CarDetectorTF
detector = CarDetectorTFV2()
#detector = CarDetectorTF()

error loading urlib2. Python2/3 mismatch?


## 2) Perform object detection on kitti video

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%matplotlib notebook

import cv2
import pykitti
from PIL import Image

date = '2011_09_26'
#drive = '0001'
drive = '0015'
basedir = 'data'
v2c_filepath = './data/2011_09_26/calib_velo_to_cam.txt'
c2c_filepath = './data/2011_09_26/calib_cam_to_cam.txt'
dataset = pykitti.raw(basedir, date, drive)


In [4]:
from source import parseTrackletXML as xmlParser

def load_tracklets_for_frames(n_frames, xml_path):
    """
    Loads dataset labels also referred to as tracklets, saving them individually for each frame.

    Parameters
    ----------
    n_frames    : Number of frames in the dataset.
    xml_path    : Path to the tracklets XML.

    Returns
    -------
    Tuple of dictionaries with integer keys corresponding to absolute frame numbers and arrays as values. First array
    contains coordinates of bounding box vertices for each object in the frame, and the second array contains objects
    types as strings.
    """
    tracklets = xmlParser.parseXML(xml_path)

    frame_tracklets = {}
    frame_tracklets_types = {}
    for i in range(n_frames):
        frame_tracklets[i] = []
        frame_tracklets_types[i] = []

    # loop over tracklets
    for i, tracklet in enumerate(tracklets):
        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        # in velodyne coordinates around zero point and without orientation yet
        trackletBox = np.array([
            [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
            [0.0, 0.0, 0.0, 0.0, h, h, h, h]
        ])
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:
            # determine if object is in the image; otherwise continue
            if truncation not in (xmlParser.TRUNC_IN_IMAGE, xmlParser.TRUNC_TRUNCATED):
                continue
            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]  # other rotations are supposedly 0
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([
                [np.cos(yaw), -np.sin(yaw), 0.0],
                [np.sin(yaw), np.cos(yaw), 0.0],
                [0.0, 0.0, 1.0]
            ])
            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8, 1)).T
            frame_tracklets[absoluteFrameNumber] = frame_tracklets[absoluteFrameNumber] + [cornerPosInVelo]
            frame_tracklets_types[absoluteFrameNumber] = frame_tracklets_types[absoluteFrameNumber] + [
                tracklet.objectType]

    return (frame_tracklets, frame_tracklets_types)
tracklet_rects, tracklet_types = load_tracklets_for_frames(len(list(dataset.velo)), 'data/{}/{}_drive_{}_sync/tracklet_labels.xml'.format(date, date, drive))

Parsing tracklet file data/2011_09_26/2011_09_26_drive_0015_sync/tracklet_labels.xml
File contains 36 tracklets
Loaded 36 tracklets.


### Loop through cam2 files and perform detection

#### First, copy all necessary KITTI helper functions from eda notebook:

In [5]:
def calib_velo2cam(filepath):
    """ 
    get Rotation(R : 3x3), Translation(T : 3x1) matrix info 
    using R,T matrix, we can convert velodyne coordinates to camera coordinates
    """
    with open(filepath, "r") as f:
        file = f.readlines()    
        
        for line in file:
            (key, val) = line.split(':',1)
            if key == 'R':
                R = np.fromstring(val, sep=' ')
                R = R.reshape(3, 3)
            if key == 'T':
                T = np.fromstring(val, sep=' ')
                T = T.reshape(3, 1)
    return R, T

def calib_cam2cam(filepath, mode):
    """
    If your image is 'rectified image' :
        get only Projection(P : 3x4) matrix is enough
    but if your image is 'distorted image'(not rectified image) :
        you need undistortion step using distortion coefficients(5 : D)
        
    in this code, I'll get P matrix since I'm using rectified image
    """
    with open(filepath, "r") as f:
        file = f.readlines()
        
        for line in file:
            (key, val) = line.split(':',1)
            if key == ('P_rect_' + mode):
                P_ = np.fromstring(val, sep=' ')
                P_ = P_.reshape(3, 4)
                # erase 4th column ([0,0,0])
                P_ = P_[:3, :3]
    return P_
 
def velo3d_2_camera2d_points(points, v_fov, h_fov, vc_path, cc_path, mode='02'):
    """ print velodyne 3D points corresponding to camera 2D image """
    # R_vc = Rotation matrix ( velodyne -> camera )
    # T_vc = Translation matrix ( velodyne -> camera )
    R_vc, T_vc = calib_velo2cam(vc_path)
    
    # P_ = Projection matrix ( camera coordinates 3d points -> image plane 2d points )
    P_ = calib_cam2cam(cc_path, mode)

    """
    xyz_v - 3D velodyne points corresponding to h, v FOV in the velodyne coordinates
    c_    - color value(HSV's Hue) corresponding to distance(m)
    
             [x_1 , x_2 , .. ]
    xyz_v =  [y_1 , y_2 , .. ]   
             [z_1 , z_2 , .. ]
             [ 1  ,  1  , .. ]
    """  
    xyz_v, c_ = velo_points_filter(points, v_fov, h_fov)
    
    """
    RT_ - rotation matrix & translation matrix
        ( velodyne coordinates -> camera coordinates )
    
            [r_11 , r_12 , r_13 , t_x ]
    RT_  =  [r_21 , r_22 , r_23 , t_y ]   
            [r_31 , r_32 , r_33 , t_z ]
    """
    RT_ = np.concatenate((R_vc, T_vc),axis = 1)
    
    # convert velodyne coordinates(X_v, Y_v, Z_v) to camera coordinates(X_c, Y_c, Z_c) 
    for i in range(xyz_v.shape[1]):
        xyz_v[:3,i] = np.matmul(RT_, xyz_v[:,i])
        
    """
    xyz_c - 3D velodyne points corresponding to h, v FOV in the camera coordinates
             [x_1 , x_2 , .. ]
    xyz_c =  [y_1 , y_2 , .. ]   
             [z_1 , z_2 , .. ]
    """ 
    xyz_c = np.delete(xyz_v, 3, axis=0)

    # convert camera coordinates(X_c, Y_c, Z_c) image(pixel) coordinates(x,y) 
    for i in range(xyz_c.shape[1]):
        xyz_c[:,i] = np.matmul(P_, xyz_c[:,i])    

    """
    xy_i - 3D velodyne points corresponding to h, v FOV in the image(pixel) coordinates before scale adjustment
    ans  - 3D velodyne points corresponding to h, v FOV in the image(pixel) coordinates
             [s_1*x_1 , s_2*x_2 , .. ]
    xy_i =   [s_1*y_1 , s_2*y_2 , .. ]        ans =   [x_1 , x_2 , .. ]  
             [  s_1   ,   s_2   , .. ]                [y_1 , y_2 , .. ]
    """
    xy_i = xyz_c[::]/xyz_c[::][2]
    ans = np.delete(xy_i, 2, axis=0)
    
    """
    width = 1242
    height = 375
    w_range = in_range_points(ans[0], width)
    h_range = in_range_points(ans[1], height)

    ans_x = ans[0][np.logical_and(w_range,h_range)][:,None].T
    ans_y = ans[1][np.logical_and(w_range,h_range)][:,None].T
    c_ = c_[np.logical_and(w_range,h_range)]

    ans = np.vstack((ans_x, ans_y))
    """
    
    return ans, c_
def depth_color(val, min_d=0, max_d=120):
    """ 
    print Color(HSV's H value) corresponding to distance(m) 
    close distance = red , far distance = blue
    """
    np.clip(val, 0, max_d, out=val) # max distance is 120m but usually not usual
    return (((val - min_d) / (max_d - min_d)) * 120).astype(np.uint8) 

def in_h_range_points(points, m, n, fov):
    """ extract horizontal in-range points """
    return np.logical_and(np.arctan2(n,m) > (-fov[1] * np.pi / 180), \
                          np.arctan2(n,m) < (-fov[0] * np.pi / 180))

def in_v_range_points(points, m, n, fov):
    """ extract vertical in-range points """
    return np.logical_and(np.arctan2(n,m) < (fov[1] * np.pi / 180), \
                          np.arctan2(n,m) > (fov[0] * np.pi / 180))

def fov_setting(points, x, y, z, dist, h_fov, v_fov):
    """ filter points based on h,v FOV  """
    
    if h_fov[1] == 180 and h_fov[0] == -180 and v_fov[1] == 2.0 and v_fov[0] == -24.9:
        return points
    
    if h_fov[1] == 180 and h_fov[0] == -180:
        return points[in_v_range_points(points, dist, z, v_fov)]
    elif v_fov[1] == 2.0 and v_fov[0] == -24.9:        
        return points[in_h_range_points(points, x, y, h_fov)]
    else:
        h_points = in_h_range_points(points, x, y, h_fov)
        v_points = in_v_range_points(points, dist, z, v_fov)
        return points[np.logical_and(h_points, v_points)]

def in_range_points(points, size):
    """ extract in-range points """
    return np.logical_and(points > 0, points < size)    

def velo_points_filter(points, v_fov, h_fov):
    """ extract points corresponding to FOV setting """
    
    # Projecting to 2D
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    dist = np.sqrt(x ** 2 + y ** 2 + z ** 2)

    if h_fov[0] < -90:
        h_fov = (-90,) + h_fov[1:]
    if h_fov[1] > 90:
        h_fov = h_fov[:1] + (90,)
    
    x_lim = fov_setting(x, x, y, z, dist, h_fov, v_fov)[:,None]
    y_lim = fov_setting(y, x, y, z, dist, h_fov, v_fov)[:,None]
    z_lim = fov_setting(z, x, y, z, dist, h_fov, v_fov)[:,None]

    # Stack arrays in sequence horizontally
    xyz_ = np.hstack((x_lim, y_lim, z_lim))
    xyz_ = xyz_.T

    # stack (1,n) arrays filled with the number 1
    one_mat = np.full((1, xyz_.shape[1]), 1)
    xyz_ = np.concatenate((xyz_, one_mat),axis = 0)

    # need dist info for points color
    dist_lim = fov_setting(dist, x, y, z, dist, h_fov, v_fov)
    color = depth_color(dist_lim, 0, 70)
    
    return xyz_, color

#### Define some helper functions of our own:

In [6]:
def draw_detections(detections: list, img: np.ndarray, confidence_threshold:float, color=(0,255,0)):
    """
    Draw detections to an image
    Args:
        detections: list of bounding boxes [[x1, y1, x2, y2], ..., ...]
        img: image to draw detections to
        confidence_threshold: (0.0 to 1.0) only draw detections above this threshold
        color: (Optional) provide a color to draw the bounding boxes
        
    """
    bboxes, class_names, confidences = detections
    for bbox, class_name, confidence in zip(bboxes, class_names, confidences):
        #if class_name.lower() in ['car', 'truck', 'bus', 'van'] and confidence > confidence_threshold:
        if True: # NOTE: just draw everything
            x1, y1, x2, y2 = bbox
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=2)
            text = '{} {:.1f}%'.format(class_name,
                                           confidence * 100)
            cv2.putText(img, text, (x1, y1-5), 1, 1.5, color, 2)

def tracklets_to_bbox_labels(frame_tracklets, frame_labels):
    """
    Convert 3d tracklets to 2D bounding boxes with corresponding labels
    Args:
        frame_tracklets: list of trackles for given frame
        frame_labels: list of labels for given frame
    Returns:
        tuple: (bboxes, labels, confidences) for given frame
    """
    bboxes, labels, confidences = [], [], []
    for single_tracklet, single_label in zip(frame_tracklets, frame_labels):
        single_tracklet_px, c_ = velo3d_2_camera2d_points(single_tracklet.T, v_fov=(-24.9, 2.0), h_fov=(-45,45), \
                                       vc_path=v2c_filepath, cc_path=c2c_filepath, mode='02')
        single_tracklet_px = single_tracklet_px.astype(int).T
        prev_pnt = single_tracklet_px[0]
        x1 = single_tracklet_px[:, 0].min()
        y1 = single_tracklet_px[:, 1].min()
        x2 = single_tracklet_px[:, 0].max()
        y2 = single_tracklet_px[:, 1].max()
        
        x_min_world = single_tracklet.T[:, 0].min()
        y_avg_world = single_tracklet.T[:, 1].mean()
        world_dist = np.sqrt(x_min_world**2 + y_avg_world**2)
        bboxes.append([x1, y1, x2, y2])
        labels.append(single_label)
        confidences.append(1.0)
    return bboxes, labels, confidences

In [7]:
from tqdm import tqdm_notebook
all_detections, all_ground_truths = [], []
PAUSE = False
for frame_number in tqdm_notebook(range(len(dataset.cam2_files))):
    img = np.array(dataset.get_cam2(frame_number))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detections = detector.detect(img=img, return_class_scores=True)
    all_detections.append(detections)
    draw_detections(detections=detections, img=img, confidence_threshold=0.8, color=(0, 0, 255))
    frame_tracklets = tracklet_rects[frame_number]
    frame_labels = tracklet_types[frame_number]
    detections_ground_truth = tracklets_to_bbox_labels(frame_tracklets, frame_labels)
    all_ground_truths.append(detections_ground_truth)
    draw_detections(detections=detections_ground_truth, img=img, confidence_threshold=0, color=(0,255,0))
    cv2.imshow('image', img)
    key = cv2.waitKey(1)
    if key & 0xFF == 27 or key == ord('q'):
        break
    elif key == ord('p'):
        PAUSE = not PAUSE
    while PAUSE is True:
        cv2.imshow('image', img)
        key = cv2.waitKey(1)
        if key == ord('p'):
            PAUSE = not PAUSE
cv2.destroyAllWindows()

## 3) Evaluate accuracy of detections against kitti ground truth labels for given video.

### Intersection over union (IOU) between two bounding boxes
Use this blog as reference:https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/  
```
Intersection over Union is an evaluation metric used to measure the accuracy of an object detector on a particular dataset. We often see this evaluation metric used in object detection challenges such as the popular PASCAL VOC challenge.
```

In [8]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

### Quick example of calculating IOU for predicted vs. Ground Truth
NOTE: ground truth and predicted coords taken directly from video by mouse cursor

In [9]:
box_pred = [573, 176, 640, 224]
box_gt = [588, 175, 643, 228]

example_iou = bb_intersection_over_union(box_pred, box_gt)
print("PRED BBOX HEIGHT = {}".format(box_pred[3] - box_pred[1]))
print("GT BBOX HEIGHT = {}".format(box_gt[3] - box_gt[1]))

print("IOU = {}".format(example_iou))

PRED BBOX HEIGHT = 48
GT BBOX HEIGHT = 53
IOU = 0.6908752327746741


### First check one frame worth of detected bounding boxes versus ground truth

In [10]:
detections_1 = all_detections[1][0]
ground_truth_1 = all_ground_truths[1][0]

In [11]:
print(detections_1)
print(ground_truth_1)

[[361, 178, 384, 192], [279, 181, 312, 199], [576, 175, 602, 204], [555, 164, 567, 174]]
[[246, 173, 300, 202], [320, 175, 356, 196], [577, 176, 613, 212]]


In [12]:
det = detections_1[0]
gt = ground_truth_1[0]
print(det)
print(gt)
print(bb_intersection_over_union(det,gt))

[361, 178, 384, 192]
[246, 173, 300, 202]
0.0


### Extend to all detections

Determine the following:  
1) True Positives: E.g. a car is detected and overlaps with ground truth > MIN_IOU_THRESHOLD  
2) False Positives:  E.g. a car is detected and does not overlap with any ground truth at all (overlap = 0 compared to all ground truths)  
3) False Negatives: E.g. a car is not detected but there exists a ground truth that has no detection assigned to it.    
These metrics will be used to generate Accuracy, Precision, and Recall  


In [13]:
results = []
true_positives, false_positives, false_negatives = [], [], []
MIN_IOU_THRESHOLD = 0.70  # as per KITTI standards
MIN_FALSE_NEGATIVE_IOU = 0.0 # min threshold for determining whether false negative
MIN_FALSE_POSITIVE_IOU = 0.0 # min threshold for determining whether false positive
for detections, ground_truths in zip(all_detections, all_ground_truths):
    detections = detections[0]   # bboxes at index 0 (assume labels are correct, for now)
    ground_truths = ground_truths[0]  
    pass_fail = []
    # calculate true positive and false negatives
    for gt in ground_truths:
        false_negative = True
        greatest_iou = 0.0
        for det in detections:
            iou = bb_intersection_over_union(gt, det)
            if iou > greatest_iou:
                greatest_iou = iou
            if iou >= MIN_IOU_THRESHOLD:
                true_positives.append([gt, det])
            if iou > MIN_FALSE_NEGATIVE_IOU:
                false_negative = False
        if false_negative is True:
            false_negatives.append([gt, det])
    # calculate false positives
    for det in detections:
        false_positive = True
        for gt in ground_truths:
            iou = bb_intersection_over_union(gt, det)
            if iou > MIN_FALSE_POSITIVE_IOU:
                false_positive = False
        if false_positive is True:
            false_positives.append([det, ground_truths])

precision = len(true_positives) / (len(true_positives) + len(false_positives))
recall = len(true_positives) / (len(true_positives) + len(false_negatives))
print("number of true positives: {}".format(len(true_positives)))
print("number of false positives: {}".format(len(false_positives)))
print("number of false negatives: {}".format(len(false_negatives)))
print("PRECISION = {}".format(precision))
print("RECALL = {}".format(recall))

number of true positives: 161
number of false positives: 396
number of false negatives: 598
PRECISION = 0.289048473967684
RECALL = 0.21212121212121213


### Quick comparison of our v1 detector vs. v2 detector
#### V1 detector (mobilenet):   
number of true positives: 161  
number of false positives: 396  
number of false negatives: 598  
PRECISION = 0.289048473967684  
RECALL = 0.21212121212121213  
#### V2 detector (YoloV3):  
number of true positives: 379  
number of false positives: 39  
number of false negatives: 400  
PRECISION = 0.9066985645933014  
RECALL = 0.4865211810012837  